---

Feature Engineering

---

_The assertions and methodologies outlined in this notebook are substantiated by referenced scientific studies detailed in the README file._

### Load libraries and Data

In [1]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import sys
sys.path.append("../src")
from extract_features import *
from visualize_audio import *
from data_manager import *
from preprocess_data import *
from load_config import *

In [2]:
constants = load_constants_from_yaml('../constants.yml')

SAMPLING_RATING = constants["SAMPLING_RATING"]
FRAME_LENGTH_ENERGY = constants["FRAME_LENGTH_ENERGY"]
THRESHOLD_PERCENTAGE = constants["THRESHOLD_PERCENTAGE"]
MIN_SILENCE_DURATION = constants["MIN_SILENCE_DURATION"]
HOP_LENGTH = constants["HOP_LENGTH"]
SEGMENT_DURATION = constants["SEGMENT_DURATION"]
SEGMENT_OVERLAP = constants["SEGMENT_OVERLAP"]
N_MFCC = constants["N_MFCC"]
CONSIDERED_ACCENTS = constants["CONSIDERED_ACCENTS"]

In [3]:
audio_data, labels = load_audio_files("../data/raw/recordings/", sr=SAMPLING_RATING)
audio_data,labels = filter_data_based_on_accents(audio_data=audio_data, labels=labels, considered_accents=CONSIDERED_ACCENTS)

Trim silence from audio

In [4]:
audio_data_trimmed, labels_trimmed = split_all_audios_by_silence(
    audio_data=audio_data,
    labels=labels,
    threshold_percentage=THRESHOLD_PERCENTAGE,
    sampling_rating=SAMPLING_RATING,
    min_silence_duration=MIN_SILENCE_DURATION
    )

One of the most commonly used spectral feature representations is the Mel-frequency cepstral coefficients (MFCC). MFCC features are generally employed in automatic speech recognition (ASR) and accent recognition systems and are known to perform best in shallow models. Spectrograms, on the other hand, are more effective in deep models and are sometimes utilized in accent recognition. We will extract MFCCs using the Librosa library.

In [6]:
# extract MFCC features from trimmed audio data (not on segmented audio data because the function itself will split the audio data into segments)
mfccs = []
for audio in audio_data_trimmed:
    mfcc = compute_mfcc(
        audio, SAMPLING_RATING, n_mfcc=N_MFCC, duration=SEGMENT_DURATION, overlap=SEGMENT_OVERLAP
    )
    mfccs.append(mfcc)

KeyboardInterrupt: 

In [ ]:
print(f"len(mfccs): {len(mfccs)}")
print(f"num_segments for mfccs[0]: {mfccs[0].shape[1]}")
print(f"num_segments for mfccs[5]: {mfccs[5].shape[1]}")

len(mfccs): 385
num_segments for mfccs[0]: 2318
num_segments for mfccs[5]: 2321


In [ ]:
# we want to split the mfccs into segments with the corresponding labels

# Initialize lists to store segments and their corresponding labels
segmented_mfccs = []
segmented_labels = []

# Iterate over each audio
for i, mfcc in enumerate(mfccs):
    # Get the shape of MFCCs and label for the current audio
    mfcc_shape = mfcc.shape
    label = labels_trimmed[i]

    # Extract the number of segments and the number of frames per segment
    num_segments = mfcc.shape[1]  # Second dimension of the MFCC shape

    # Iterate over each segment in the current audio
    for j in range(num_segments):
        # Extract the MFCCs for the current segment
        mfcc_segment = mfccs[i][:, j]

        # Append the segment and its corresponding label to the lists
        segmented_mfccs.append(mfcc_segment)
        segmented_labels.append(label)

# Convert the lists to NumPy arrays
# segmented_mfccs = np.array(segmented_mfccs)
# segmented_labels = np.array(segmented_labels)

print(f"len(segmented_mfccs): {len(segmented_mfccs)}")
print(f"len(segmented_labels): {len(segmented_labels)}")
print(f"segmented_mfccs[0]: {len(segmented_mfccs[0])}")
print(f"segmented_labels[0]: {segmented_labels[0]}")

len(segmented_mfccs): 941351
len(segmented_labels): 941351
segmented_mfccs[0]: 13
segmented_labels[0]: english
